In [5]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
#import pyLDAvis.gensim_models
#from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import json
import jsonlines
from collections import Counter
from tqdm.notebook import tqdm
import gc

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet', 'omw-1.4'])

#warning
import warnings 
#from PyPDF2 import PdfFileReader
from os.path import join
from tqdm.notebook import tqdm
import os
import argostranslate.package
import argostranslate.translate

tqdm.pandas()

warnings.filterwarnings('ignore')
#DATA_REK = 'Data'
#PRO_FILES = '/content/drive/MyDrive/PhD/Yousra_Data'
DATA_FILES = 'Data/SkillNERV2'

DATA_REK = 'Data/Rekrute'

ACC_DIR = 'data/transformed_data/acceptability'
SKILL_OUT = 'data/transformed_data/extracted_skills'


[nltk_data] Downloading package stopwords to /home/vortex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vortex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vortex/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

## Sentence Translation

In [7]:
def write_json_lines(file_name,dict_data):
    json_string = json.dumps(dict_data)
    with open(file_name, 'a') as f:
        f.write(json_string+"\n")
        
def read_json_lines(file_name):
    lines = []
    with open(file_name) as file_in:
        for line in file_in:
            lines.append(json.loads(line))
    return lines

In [8]:
from_code = "fr"
to_code = "en"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
package_to_install = next(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)

header_written = False
argostranslate.package.install_from_path(package_to_install.download())

### Translating skills extracted using ESCO patterns

In [9]:
#df.to_csv(join(SKILL_OUT, "fr_extracted_skills.csv"), index=False)

In [10]:
#df = pd.read_csv(join(SKILL_OUT, 'fr_extracted_skills.csv'))

In [11]:
#from translate import Translator
#translator= Translator(from_lang="fr", to_lang="en")

In [20]:
def translate_skills(df_skills, file_name, id_col = 'job_offer_id'):
    if os.path.exists(file_name):
        exist_df = pd.DataFrame(read_json_lines(file_name))
        print('all skiils', df_skills.shape[0])
        df_skills = df_skills[~df_skills[id_col].isin(exist_df['id'])]
        print('rest of skills', df_skills.shape[0])
    skills_en = []
    ttrans = argostranslate.translate
    for ind, row in tqdm(df_skills.iterrows(), total=df_skills.shape[0]):
        skills = row['skills'].split('|')
        rows = {}
        en_skills = [ttrans.translate(i, from_code, to_code) for i in skills]
        gc.collect()
        #skills_en.append('|'.join(en_skills))
        rows['id'] = row[id_col]
        rows['en_skills'] = en_skills
        write_json_lines(file_name,rows)

### Job Offers: Translating skills extracted using ESCO extracted patterns

In [13]:
#df = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'fr_extracted_skills_v3.jsonl')))

In [14]:
#df

In [15]:
#translate_skills(df, join(ACC_DIR, 'en_skills_ESCO_argos.jsonl'))
####

### Job Offers: Translating skills extracted using Pole Emploi extracted patterns

In [16]:
df_pe = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'extracted_skills_pole_emploi_v2.jsonl')))

In [17]:
translate_skills(df_pe, join(ACC_DIR, 'en_skills_pole_emploi_argos.jsonl'))

  0%|          | 0/123991 [00:00<?, ?it/s]

### Coursera: Translating skills extracted using  ESCO extracted patterns

In [23]:
df_co = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'fr_coursera_extracted_skills_v2.jsonl')))

In [24]:
translate_skills(df_co, join(ACC_DIR, 'en_skills_coursera_argos.jsonl'), 'id')

all skiils 10180
rest of skills 9341


  0%|          | 0/9341 [00:00<?, ?it/s]

### Coursera: Translating skills extracted using  Pol Emploi extracted patterns

In [25]:
df_co = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'fr_coursera_extracted_skills_pole_emploi_v1.jsonl')))

In [26]:
translate_skills(df_co, join(ACC_DIR, 'en_skills_coursera_PE_argos.jsonl'), 'id')

  0%|          | 0/10180 [00:00<?, ?it/s]